# Homework 4

## Question 2: Applied ML


### Loading the data
First we load the data and vectorize it.
The library functions contained in sklearn make this very straightforward.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# we use the builtin function for loading data
# sklearn already has a split in train/test, you can specify which data you want with the "subset" parameter
# since we will perform that split ourselves, we load all data
# we also remove all metadata

from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(subset="all")

In [ ]:
# the newsgroups are an sklearn "bunch"
# it resembles a dictionary
newsgroups.keys()

In [ ]:
# data contains the text for each article
newsgroups.data[0]

In [ ]:
# check how many articles we have
len(newsgroups.data)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# this will create a vector for every article
# the output is a matrix
matrix = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english').fit_transform(newsgroups.data)

print(type(matrix))
print(matrix.shape)

### Splitting the data

We split the data into separate sets for training, testing and evaluating.
Following the usual naming convention in machine learning, we call these datasets

 - X_train, y_train
 - X_test, y_test
 - X_val, y_val
 
where X is the data and y contains the labels

In [ ]:
# renaming
X = matrix
y = newsgroups.target

In [ ]:
# now we do the split into train, test, val
# it's 0.8, 0.1, 0.1

num_samples = len(y)
num_train = int(0.8 * num_samples)
num_test = int(0.1 * num_samples)
num_val = int(0.1 * num_samples)

X_train = X[:num_train]
X_test = X[num_train : -num_val]
X_val = X[-num_val:]


y_train = y[:num_train]
y_test = y[num_train : -num_val]
y_val = y[-num_val:]

In [ ]:
# we know whether the data is ordered
# it would be possible that we have all articles from category 1 first, then category 2, etc
# this would mean that our split is broken
# we take a look at the labels
# the scatterplot shows that there is no order, the labels don't increase linearly
plt.figure(figsize=(10,1))
plt.scatter(np.arange(len(y)), y)
plt.show()

### Fitting a RandomForest

Before we start the grid search, we fit a random forest, to see how the syntax looks.

We also want to check whether the classifier actually works, or if we have made some mistake. There are 20 categories. If the classifier is just guessing randomly, we would see an accuracy of about 1/20 = 5%. If the classifier does better than that, it is able to "learn" the data. That would mean we can continue to search for a good parametrization.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=10)
random_forest.fit(X_train, y_train)

In [ ]:
# use the classifier to predict labels for the test set
random_forest.score(X_test, y_test)

The accuracy is already rather good. And this is for top-1 results. It would probably do much better if we would compute something like the top-2 or top-3 accuracy.

The number of estimators is 10 by default. I would like to get the max depth that was used in this tree. Then we can use these parameters as the center of our grid search.

In [ ]:
# unfortunately, we are apparently not supposed to read the depth
# it is necessary to use properties with _ in their name
depths = [estimator.tree_.max_depth for estimator in random_forest.estimators_]


max_depth = max(depths)
avg_depth = sum(depths)/len(depths)

print("max depth is: ", max_depth)
print("avg depth is: ", avg_depth)

### Gridsearch, 1st approach

We use the parameters given above to set up our gridsearch.

In [ ]:
n_estimators = np.arange(10) + 5
max_depths = 7 * np.logspace(1, 2, num=5, dtype=np.int)

print("num estimators: ", n_estimators)
print("max depths: ", max_depths)

# we will use multithreading to process the grid on multiple cpus
# the load balancing of sklearn is not very complex
# it just splits the list and then collects the results
# but the parametrizations for more estimators / depths take much longer
# so we shuffle, as a simple load balancing
np.random.shuffle(n_estimators)
np.random.shuffle(max_depths)

In [ ]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from scipy.sparse import vstack

In [ ]:
train_indices = [-1] * X_train.shape[0]
test_indices = [0] * X_test.shape[0]
indices = train_indices + test_indices

X_joint = vstack([X_train, X_test])
y_joint = np.concatenate([y_train, y_test])

pds = PredefinedSplit(indices)

In [ ]:
import pickle
rerun = True

In [ ]:
if rerun:
    
    rfc = RandomForestClassifier()
    clf_grid1 = GridSearchCV(rfc, param_grid={'n_estimators':n_estimators, 'max_depth':max_depths}, cv=pds, n_jobs=-1)
    clf_grid1.fit(X_joint, y_joint)
    
    with open("data/clf_grid.pickle", "wb") as file:
        pickle.dump(clf_grid1, file)
else:
    with open("data/clf_grid.pickle", "rb") as file:
        clf_grid1 = pickle.load(file)

### Discussing results, for 1st GridSearch

The results on both the training and the test set are very good. We reach well above 90%.

But checking against the validation set shows that the results are not generalizable. We are not only overfitting the training data, the cross-validation of the parameters also resulted in training against the test set. We do note an improvement compared to the first evaluation, though.

The overfitting on the test set is so strong that we need to question whether our split is actually working. But it seems to be set up correctly.

In [ ]:
# on the train and test set, our classifier performs very well
print("train score: ", clf_grid1.score(X_train, y_train))
print("test score: ", clf_grid1.score(X_test, y_test))

In [ ]:
# but on the val set, we perform badly
print("test score: ", clf_grid1.score(X_val, y_val))

In [ ]:
# looking at the predefined splot
# it generates 1 set of train/test, that is good
print("the split generates {} splits into train and test set".format(pds.get_n_splits()))

# let's check the indices
# they also look good
train, test = next(pds.split())
print("the training indices go from {} to {}".format(train.min(), train.max()))
print("the testing indices go from {} to {}".format(test.min(), test.max()))

### Gridsearch, 2nd approach
Our gridsearch is strongly overfitting.

Overfitting is usually due to too high model complexity. So we start a second gridsearch with reduced complexity.

In [ ]:
n_estimators2 = n_estimators = np.arange(10) + 5
max_depths2 = 2 * np.logspace(1, 2, num=5, dtype=np.int)


print("num estimators: ", n_estimators2)
print("max depths: ", max_depths2)

np.random.shuffle(n_estimators2)
np.random.shuffle(max_depths2)

In [ ]:
if rerun:
    
    rfc = RandomForestClassifier()
    clf_grid2 = GridSearchCV(rfc, param_grid={'n_estimators':n_estimators2, 'max_depth':max_depths2}, cv=pds, n_jobs=-1)
    clf_grid2.fit(X_joint, y_joint)
    
    with open("data/clf_grid2.pickle", "wb") as file:
        pickle.dump(clf_grid2, file)
else:
    with open("data/clf_grid2.pickle", "rb") as file:
        clf_grid2 = pickle.load(file)

### Discussing results for the 2nd gridsearch

The performance is very similar to the first gridsearch

In [ ]:
# on the train and test set, our classifier performs very well
print("train score: ", clf_grid2.score(X_train, y_train))
print("test score: ", clf_grid2.score(X_test, y_test))

In [ ]:
# but on the val set, we perform badly
print("test score: ", clf_grid2.score(X_val, y_val))

### Revisiting the baseline
Between the first and second gridsearch, we changed the maximum depth. Which had only very little effect on the performance.

We could also change the number of estimators, but this has a strong effect on the complexity and increases the time for training drastically.

Therefore, we do a simple check first: We train just one random forest, with a higher number of estimators and evaluate its performance.

In [ ]:
rf2 = RandomForestClassifier(n_estimators=100)
rf2.fit(X_train, y_train)

In [ ]:
rf2.score(X_train, y_train)

In [ ]:
rf2.score(X_test, y_test)

In [ ]:
depths = [estimator.tree_.max_depth for estimator in rf2.estimators_]


max_depth = max(depths)
avg_depth = sum(depths)/len(depths)

print("max depth is: ", max_depth)
print("avg depth is: ", avg_depth)

### Gridsearch for the 3rd time

Increasing the number of estimators improves both the training and the validation accuracy.
So we need to perform a gridsearch for higher amounts of estimators.
This will need a lot of time.
We set the gridsearch up to sample many configurations and let it run over night.

In [ ]:
n_estimators3 = (40 * np.logspace(0, 1.5, num=20)).astype(np.int)
max_depths3 = 250 + (np.arange(10)*50)

print("num estimators: ", n_estimators3)
print("max depths: ", max_depths3)

np.random.shuffle(n_estimators3)
np.random.shuffle(max_depths3)

In [ ]:
if rerun:
    
    rfc = RandomForestClassifier()
    clf_grid3 = GridSearchCV(rfc, param_grid={'n_estimators':n_estimators3, 'max_depth':max_depths3}, cv=pds, n_jobs=-1)
    clf_grid3.fit(X_joint, y_joint)
    
    with open("data/clf_grid3.pickle", "wb") as file:
        pickle.dump(clf_grid3, file)
else:
    with open("data/clf_grid3.pickle", "rb") as file:
        clf_grid3 = pickle.load(file)

### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix